In [1]:
import xarray as xr
import matplotlib.pyplot as plt
import numpy as np
import sys
sys.path.append("/home/qwertz/Dokumente/N_eigene_Dokumente/Geo_Met_MSc/Semester_4/Thesis_prep_Juni_23/Porject_Work/")
from Sc_module import convert_single_timestamp
plt.rc('font', size=18) 
plt.style.use('seaborn-poster')

outputdir = "/home/qwertz/Dokumente/N_eigene_Dokumente/Geo_Met_MSc/Semester_4_5/Thesis_prep_Juni_23/MA/master_plots/22_profiles/"

stationfiles = ["/home/qwertz/data_js_icon_22_23_August_2018/stat_11_all-times.nc",\
                "/home/qwertz/data_js_icon_22_23_August_2018/stat_20_all-times.nc",\
                "/home/qwertz/data_js_icon_22_23_August_2018/stat_21_all-times.nc",\
                "/home/qwertz/data_js_icon_22_23_August_2018/stat_IQQ_all-times.nc"]
file_for_heights = "/home/qwertz/data_js_icon_22_23_August_2018/cloud_DOM01.20180822.selvar.int.nc"

##################################################

variable_list = ["pres", "temp", "theta_v", "rh", "qv", "qc", "qr"]
timesteps = [50]
#timesteps = [50, 75, 100, 125, 150, 175, 200, 225, 250, 275, 300] # geht nur bis 3xx für diese Profile...

#date_list = [convert_single_timestamp(date) for date in ds["time"].values]
#for i,date in enumerate(date_list):
#    if i%10==0:
#        print(i,date)
    
############################

for file in stationfiles:
    ds = xr.open_dataset(file, engine="netcdf4")
    for timestep in timesteps:
        for k,variable in enumerate(variable_list):
            #var = variable
            station = file.split("/")[-1].split("_")[1]
            date_string = str(convert_single_timestamp(ds["time"].isel(time=timestep).values))
            any_heights = ds["z_mc"].isel(ncells=0).values
            plt.figure(figsize=(15,8))
            plt.plot(ds[variable].isel(time=timestep).isel(ncells=0).values[150:], any_heights[150:])
            plt.ylabel("height [m]")
            plt.xlabel(variable+" in ["+ds[variable].units+"]")
            plt.title("Profile of "+variable+" at station "+station+"\n"+date_string)
            plt.ylim(0,2500)
            plt.tight_layout()
            plt.savefig(outputdir+str(timestep)+variable+"_"+station+"_profile.png")
            plt.close()


/usr/lib/python3/dist-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
# Humidity Profiles from measurement (foghat / iqq):

import xarray as xr
import matplotlib.pyplot as plt
import numpy as np
from scipy.interpolate import interp1d
import sys
sys.path.append("/home/qwertz/Dokumente/N_eigene_Dokumente/Geo_Met_MSc/Semester_4/Thesis_prep_Juni_23/Porject_Work/")
from Sc_module import convert_single_timestamp

#################################
from Sc_module import magnusformel
from Sc_module import clausius_clapeyron
from Sc_module import absolute_moisture_rhov_or_f
from Sc_module import vapour_pressure_e_from_rh
################################

file_icon = "/home/qwertz/data_js_icon_22_23_August_2018/stat_IQQ_all-times.nc"
file_hua22 = "/home/qwertz/obs_data_MA/iqq_foghat/22/sups_iqq_mwr00_l2_hua_p00_20180822000000.nc"
file_prw22 = "/home/qwertz/obs_data_MA/iqq_foghat/22/sups_iqq_mwr00_l2_prw_p00_20180822000000.nc"
file_00ta22 = "/home/qwertz/obs_data_MA/iqq_foghat/22/sups_iqq_mwr00_l2_ta_p00_20180822000000.nc"
file_BLta22 = "/home/qwertz/obs_data_MA/iqq_foghat/22/sups_iqq_mwrBL00_l2_ta_p00_20180822000243.nc"
file_hua23 = "/home/qwertz/obs_data_MA/iqq_foghat/23/sups_iqq_mwr00_l2_hua_p00_20180823000000.nc"
file_prw23 = "/home/qwertz/obs_data_MA/iqq_foghat/23/sups_iqq_mwr00_l2_prw_p00_20180823000000.nc"
file_00ta23 = "/home/qwertz/obs_data_MA/iqq_foghat/23/sups_iqq_mwr00_l2_ta_p00_20180823000000.nc"
file_BLta23 = "/home/qwertz/obs_data_MA/iqq_foghat/23/sups_iqq_mwrBL00_l2_ta_p00_20180823000753.nc"

ds_icon = xr.open_dataset(file_icon)
ds_hua22 = xr.open_dataset(file_hua22)
ds_prw22 = xr.open_dataset(file_prw22)
ds_00ta22 = xr.open_dataset(file_00ta22)
ds_BLta22 = xr.open_dataset(file_BLta22)
ds_hua23 = xr.open_dataset(file_hua23)
ds_prw23 = xr.open_dataset(file_prw23)
ds_00ta23 = xr.open_dataset(file_00ta23)
ds_BLta23 = xr.open_dataset(file_BLta23)
ds_hua = xr.concat([ds_hua22, ds_hua23], dim="time") # absolute humidity kg m-3 rho
ds_prw = xr.concat([ds_prw22, ds_prw23], dim="time") # atmosphere_mass_content_of_water_vapor kg m-2 IWV
ds_00ta = xr.concat([ds_00ta22, ds_00ta23], dim="time") # T Kelvin
ds_BLta = xr.concat([ds_BLta22, ds_BLta23], dim="time") # t Kelvin / less timesteps...
#####
# Entfernen von doppelten Zeitpunkten:
ds_hua = ds_hua.drop_duplicates("time")
ds_prw = ds_prw.drop_duplicates("time")
ds_00ta = ds_00ta.drop_duplicates("time")
ds_BLta = ds_BLta.drop_duplicates("time")

################################

def magnusformel(temp_celsius):
    # Sättigungsdampfdruck für eine Temperatur in °C
    # es returned in Pa
    es = 610.78 * np.exp((17.08085 * temp_celsius)/(243.175 + temp_celsius))
    return es

#################################

def clausius_clapeyron(temp_celsius):
    # Sättigungsdampfdruck für eine Temperatur in °C
    # es returned in Pa
    es = 610.78 * np.exp(2.5e6 / 462 * (1/273.15 - 1/(273.15+temp_celsius)))
    return es

#################################

def absolute_moisture_rhov_or_f(vapour_pressure_e, temperature_abs):
    # calculates kg/m**3 from %
    # Takes in e_pressure in Pa
    Rv = 461.52
    rhov = vapour_pressure_e / (temperature_abs * Rv)
    return rhov

################################

def vapour_pressure_e_from_rh(relative_humidity=70, temperature_abs=273.15-15):
    # e returned in hPa
    vapour_pressure_e = clausius_clapeyron(temperature_abs-273.15) * relative_humidity / 100

    #######
    '''
    if abs(clausius_clapeyron(temperature_abs-273.15)-magnusformel(temperature_abs-273.15))/magnusformel(temperature_abs-273.15)>0.05:
        print("Abweichung um mehr als 5 % zwischen CC und Magnus...")
    '''
    #######
        
    return vapour_pressure_e

################################
#############################################
# Calculate new humidity values:

def print_all_hum_plots_for_timeperiod(begin_float=20180822, end_float=20180822.25):
    datetime_begin = str(convert_single_timestamp(begin_float))[:19]
    datetime_end = str(convert_single_timestamp(end_float))[:19]
    
    # (time, height, ncells) 
    # 0- 6 h ==20180822, 20180822.25
    ds_icon_by_height = ds_icon.sel(time=slice(begin_float, end_float)).mean(dim="time").isel(ncells=0)
    icon_heights = ds_icon["z_mc"].isel(ncells=0).values
    abs_hum_icon = []
    for i, (tempi, pressi, rhi) in enumerate(zip(ds_icon_by_height["temp"].values, ds_icon_by_height["pres"].values,\
                                                 ds_icon_by_height["rh"].values)):
        e_pressure = vapour_pressure_e_from_rh(rhi, tempi)
        abs_hum = absolute_moisture_rhov_or_f(e_pressure, tempi)
        abs_hum_icon.append(abs_hum)
    
    #############################################
    # Interpolate icon data on foghat heights:
    interp_func = interp1d(icon_heights, abs_hum_icon)
    interpolated_icon_hum = interp_func(ds_hua["height"].values[2:])
    
    ###############################################
    
    feuchte_iqq = ds_hua["hua"].sel(time=slice(datetime_begin, datetime_end)).mean(dim="time")
    plt.figure(figsize=(10,10))
    plt.title("Humidity in ICON and Foghat IQQ\n mean for "+\
              datetime_begin+" to "+datetime_end)
    plt.plot(feuchte_iqq, ds_hua["height"], label="humidity Foghat", color="blue")
    plt.plot(abs_hum_icon, icon_heights, label="humidity ICON", color="red")
    plt.xlabel(ds_hua["hua"].long_name+" ["+ds_hua["hua"].units+"]")
    plt.ylabel(ds_00ta["height"].standard_name+" ["+ds_hua["height"].units+"]")
    plt.ylim(0,2000)
    plt.xlim(0,0.01)
    plt.legend()
    plt.tight_layout()
    plt.savefig(outputdir+str(begin_float)+"_humidity_foghat_vs_ICON.png")
    # Draw hua error margin in here
    
    ###
    # Error margin against difference plot!
    hu_fehler_iqq = ds_hua["hua_err"].sel(time=slice(datetime_begin, datetime_end)).mean(dim="time")
    plt.figure(figsize=(10,10))
    plt.title("Humidity error in Foghat and absolute of difference ICON - Foghat\n mean for "+\
              datetime_begin+" to "+datetime_end)
    plt.plot(hu_fehler_iqq, ds_hua["height"], label="humidity error foghat", color="blue")
    plt.plot(abs(interpolated_icon_hum-feuchte_iqq[2:]), ds_hua["height"][2:],\
             label="absolute of difference ICON - Foghat", color="black")
    plt.xlabel(ds_hua["hua"].long_name+" ["+ds_hua["hua"].units+"]")
    plt.ylabel(ds_00ta["height"].standard_name+" ["+ds_hua["height"].units+"]")
    plt.ylim(0,2000)
    plt.xlim(0,0.005)
    plt.legend()
    plt.tight_layout()
    plt.savefig(outputdir+str(begin_float)+"_abs_hum_diff_and_foghat_err.png")
    
    ###
    # Difference:
    hu_fehler_iqq = ds_hua["hua_err"].sel(time=slice(datetime_begin, datetime_end)).mean(dim="time")
    plt.figure(figsize=(10,10))
    plt.title("Difference between ICON - Foghat Humidity\n mean for "+\
              datetime_begin+" to "+datetime_end)
    plt.plot(interpolated_icon_hum-feuchte_iqq[2:], ds_hua["height"][2:], label="difference ICON - Foghat", color="black")
    plt.xlabel(ds_hua["hua"].long_name+" ["+ds_hua["hua"].units+"]")
    plt.ylabel(ds_00ta["height"].standard_name+" ["+ds_hua["height"].units+"]")
    plt.ylim(0,2000)
    plt.xlim(-0.005,0.005)
    plt.legend()
    plt.tight_layout()
    plt.savefig(outputdir+str(begin_float)+"_mean_hum_diff.png")

    #############
    # Erstellen des Multipanel-Plots mit 2 nebeneinanderliegenden Subplots
    fig, axs = plt.subplots(1, 2, figsize=(16, 8))  # 1 Zeile, 2 Spalten für die Subplots
    
    # Erstes Subplot: Luftfeuchtigkeit in ICON und Foghat
    axs[0].set_title("Humidity in ICON and Foghat IQQ\n average of 22nd and 23rd of August")
    axs[0].plot(feuchte_iqq, ds_hua["height"], label="humidity Foghat", color="blue")
    axs[0].plot(abs_hum_icon, icon_heights, label="humidity ICON", color="red")
    axs[0].set_xlabel(ds_hua["hua"].long_name + " [" + ds_hua["hua"].units + "]")
    axs[0].set_ylabel(ds_00ta["height"].standard_name + " [" + ds_hua["height"].units + "]")
    axs[0].set_ylim(0, 2000)
    axs[0].set_xlim(0, 0.01)
    axs[0].legend(loc="upper right", fontsize=16)
    axs[0].grid(True)
    
    # Zweites Subplot: Fehler und Differenz der Luftfeuchtigkeit
    hu_fehler_iqq = ds_hua["hua_err"].sel(time=slice(datetime_begin, datetime_end)).mean(dim="time")
    axs[1].set_title("Humidity error in Foghat\nand absolute of difference ICON - Foghat\n average of 22nd and 23rd of August")
    axs[1].plot(hu_fehler_iqq, ds_hua["height"], label="humidity error foghat", color="blue")
    axs[1].plot(abs(interpolated_icon_hum - feuchte_iqq[2:]), ds_hua["height"][2:],\
                label="absolute of difference ICON - Foghat", color="black")
    axs[1].set_xlabel(ds_hua["hua"].long_name + " [" + ds_hua["hua"].units + "]")
    axs[1].set_ylabel(ds_00ta["height"].standard_name + " [" + ds_hua["height"].units + "]")
    axs[1].set_ylim(0, 2000)
    axs[1].set_xlim(0, 0.005)
    axs[1].legend(loc="upper right", fontsize=16)
    axs[1].grid(True)

    # print("MEan Hu error: ", np.nanmean(hu_fehler_iqq))
    # print("Level errors: ", hu_fehler_iqq)
    # print("Levels: ", ds_hua["height"])
    
    # Gesamtlayout und Speichern der Grafik
    plt.tight_layout()
    plt.savefig(outputdir + str(begin_float) + "_humidity_comparison_subplots.png")
    ############
    
    plt.close("all")
    return 0

###############################################################
t_values = [20180822.0, 20180822.25, 20180822.5, 20180822.75, 20180823.0, 20180823.25, 20180823.5, 20180823.75,\
            20180824, 20180822.0,20180824, 20180822.625, 20180822.75, 20180822.125, 20180822.25, 20180823.125,\
            20180823.25, 20180823.625,\
            20180823.75]

for i, timestep in enumerate(t_values):
    if i+1==len(t_values):
        break

    if t_values[i+1]<timestep:
        print("overleaped timestep: ", timestep, " to ", t_values[i+1])
        continue
    
    begin_float = timestep
    end_float = t_values[i+1]
    print_all_hum_plots_for_timeperiod(begin_float=begin_float, end_float=end_float)



overleaped timestep:  20180824  to  20180822.0
overleaped timestep:  20180824  to  20180822.625
overleaped timestep:  20180822.75  to  20180822.125


In [5]:
# Temperature Profiles from measurement (foghat / iqq):

import xarray as xr
import matplotlib.pyplot as plt
import numpy as np
from scipy.interpolate import interp1d

file_icon = "/home/qwertz/data_js_icon_22_23_August_2018/stat_IQQ_all-times.nc"
file_hua22 = "/home/qwertz/obs_data_MA/iqq_foghat/22/sups_iqq_mwr00_l2_hua_p00_20180822000000.nc"
file_prw22 = "/home/qwertz/obs_data_MA/iqq_foghat/22/sups_iqq_mwr00_l2_prw_p00_20180822000000.nc"
file_00ta22 = "/home/qwertz/obs_data_MA/iqq_foghat/22/sups_iqq_mwr00_l2_ta_p00_20180822000000.nc"
file_BLta22 = "/home/qwertz/obs_data_MA/iqq_foghat/22/sups_iqq_mwrBL00_l2_ta_p00_20180822000243.nc"
file_hua23 = "/home/qwertz/obs_data_MA/iqq_foghat/23/sups_iqq_mwr00_l2_hua_p00_20180823000000.nc"
file_prw23 = "/home/qwertz/obs_data_MA/iqq_foghat/23/sups_iqq_mwr00_l2_prw_p00_20180823000000.nc"
file_00ta23 = "/home/qwertz/obs_data_MA/iqq_foghat/23/sups_iqq_mwr00_l2_ta_p00_20180823000000.nc"
file_BLta23 = "/home/qwertz/obs_data_MA/iqq_foghat/23/sups_iqq_mwrBL00_l2_ta_p00_20180823000753.nc"

ds_icon = xr.open_dataset(file_icon)
ds_hua22 = xr.open_dataset(file_hua22)
ds_prw22 = xr.open_dataset(file_prw22)
ds_00ta22 = xr.open_dataset(file_00ta22)
ds_BLta22 = xr.open_dataset(file_BLta22)
ds_hua23 = xr.open_dataset(file_hua23)
ds_prw23 = xr.open_dataset(file_prw23)
ds_00ta23 = xr.open_dataset(file_00ta23)
ds_BLta23 = xr.open_dataset(file_BLta23)
ds_hua = xr.concat([ds_hua22, ds_hua23], dim="time") # absolute humidity kg m-3 rho
ds_prw = xr.concat([ds_prw22, ds_prw23], dim="time") # atmosphere_mass_content_of_water_vapor kg m-2 IWV
ds_00ta = xr.concat([ds_00ta22, ds_00ta23], dim="time") # T Kelvin
ds_BLta = xr.concat([ds_BLta22, ds_BLta23], dim="time") # t Kelvin / less timesteps...
#####
# Entfernen von doppelten Zeitpunkten:
ds_hua = ds_hua.drop_duplicates("time")
ds_prw = ds_prw.drop_duplicates("time")
ds_00ta = ds_00ta.drop_duplicates("time")
ds_BLta = ds_BLta.drop_duplicates("time")

##############################

def print_all_temp_plots_for_timeperiod(begin_float=20180822, end_float=20180822.25):
    datetime_begin = str(convert_single_timestamp(begin_float))[:19]
    datetime_end = str(convert_single_timestamp(end_float))[:19]

    ds_icon_by_height = ds_icon.sel(time=slice(begin_float, end_float)).mean(dim="time").isel(ncells=0)
    icon_heights = ds_icon["z_mc"].isel(ncells=0).values

    # Interpolate icon data on foghat heights:
    interp_func = interp1d(icon_heights, ds_icon_by_height["temp"].values)
    interpolated_icon_temp = interp_func(ds_BLta["height"].values[2:])
    #############################################

    # Temperature:
    temp_iqq1 = ds_BLta["ta"].sel(time=slice(datetime_begin, datetime_end)).mean(dim="time")
    plt.figure(figsize=(10,10))
    plt.title("Temperature in ICON and Foghat IQQ\n mean for "+\
              datetime_begin+" to "+datetime_end)
    plt.plot(temp_iqq1, ds_BLta["height"], label="temperature Foghat", color="blue")
    plt.plot(ds_icon_by_height["temp"].values, icon_heights, label="temperature ICON", color="red")
    plt.xlabel(ds_BLta["ta"].standard_name+" ["+ds_00ta["ta"].units+"]")
    plt.ylabel(ds_BLta["height"].standard_name+" ["+ds_00ta["height"].units+"]")
    plt.ylim(0,2000)
    plt.xlim(270,310)
    plt.legend()
    plt.tight_layout()
    plt.savefig(outputdir+str(begin_float)+"_temps1_foghat_vs_ICON.png")
    
    ###
    # Difference:
    err_temp_iqq1 = ds_BLta["ta_err"].sel(time=slice(datetime_begin, datetime_end)).mean(dim="time")
    plt.figure(figsize=(10,10))
    plt.title("Temperature difference ICON - Foghat and Foghat Error IQQ\n mean for "+\
              datetime_begin+" to "+datetime_end)
    plt.plot(err_temp_iqq1, ds_BLta["height"], label="temperature error foghat", color="blue")
    plt.plot(abs(interpolated_icon_temp-temp_iqq1[2:]), ds_BLta["height"].values[2:],\
             label="absolute of difference ICON - Foghat", color="black")
    plt.xlabel(ds_BLta["ta"].standard_name+" ["+ds_BLta["ta"].units+"]")
    plt.ylabel(ds_BLta["height"].standard_name+" ["+ds_BLta["height"].units+"]")
    plt.ylim(0,2000)
    plt.xlim(0,6)
    plt.legend()
    plt.tight_layout()
    plt.savefig(outputdir+str(begin_float)+"_temp_diff_and_err.png")
    
    ###
    # Temperature:
    temp_iqq2 = ds_BLta["ta"].sel(time=slice(datetime_begin, datetime_end)).mean(dim="time")
    plt.figure(figsize=(10,10))
    plt.title("Temperature in ICON and Foghat IQQ\n mean for "+\
              datetime_begin+" to "+datetime_end)
    plt.plot(temp_iqq2, ds_BLta["height"], label="temperature Foghat", color="blue")
    plt.plot(ds_icon_by_height["temp"].values, icon_heights, label="temperature ICON", color="red")
    plt.xlabel(ds_BLta["ta"].standard_name+" ["+ds_BLta["ta"].units+"]")
    plt.ylabel(ds_BLta["height"].standard_name+" ["+ds_BLta["height"].units+"]")
    plt.ylim(0,2000)
    plt.xlim(270,310)
    plt.legend()
    plt.savefig(outputdir+str(begin_float)+"_temps2_foghat_vs_ICON.png")

    ###
    # Difference:
    plt.figure(figsize=(10,10))
    plt.title("Temperature difference between ICON - Foghat \n mean for "+\
              datetime_begin+" to "+datetime_end)
    plt.plot(interpolated_icon_temp-temp_iqq2[2:], ds_BLta["height"].values[2:],\
             label="difference ICON - Foghat", color="black")

    # print("***************")
    index10 = 11
    index20 = index10-2
    # print(datetime_begin, " to ", datetime_end)
    # print("For height levels: ",ds_BLta["height"].values[2:index10])
    # print("mean diff: ", np.nanmean(interpolated_icon_temp[:index20]-temp_iqq2[2:index10]))
    # print(len(ds_BLta["height"].values[2:index10]), " == ",len(interpolated_icon_temp[:index20]-temp_iqq2[2:index10]))
    # print("***************")
    
    plt.xlabel(ds_BLta["ta"].standard_name+" ["+ds_BLta["ta"].units+"]")
    plt.ylabel(ds_BLta["height"].standard_name+" ["+ds_BLta["height"].units+"]")
    plt.ylim(0,2000)
    plt.xlim(-8,8)
    plt.legend()
    plt.tight_layout()
    plt.savefig(outputdir+str(begin_float)+"_mean_temp_diff.png")

    # Einstellungen für die Plotgröße und das Layout
    fig, axs = plt.subplots(1, 2, figsize=(16, 8))  # 1 Zeile, 2 Spalten für die Subplots
    
    # Erstes Subplot: Temperaturdifferenz und Fehler
    err_temp_iqq1 = ds_BLta["ta_err"].sel(time=slice(datetime_begin, datetime_end)).mean(dim="time")
    axs[1].set_title("Temperature difference ICON - Foghat\nand Foghat Error IQQ\n average of 22nd and 23rd of August")
    axs[1].plot(err_temp_iqq1, ds_BLta["height"], label="temperature error foghat", color="blue")
    axs[1].plot(abs(interpolated_icon_temp - temp_iqq1[2:]), ds_BLta["height"].values[2:], label="absolute of difference ICON - Foghat", color="black")
    axs[1].set_xlabel(ds_BLta["ta"].standard_name + " [" + ds_BLta["ta"].units + "]")
    axs[1].set_ylabel(ds_BLta["height"].standard_name + " [" + ds_BLta["height"].units + "]")
    axs[1].set_ylim(0, 2000)
    axs[1].set_xlim(0, 6)
    axs[1].legend(loc="upper right", fontsize=16)
    axs[1].grid(True)

    # print("MEan T error: ", np.nanmean(err_temp_iqq1))
    # print("Level errors: ", err_temp_iqq1)
    # print("Levels: ", ds_BLta["height"])
    
    # Zweites Subplot: Temperatur in ICON und Foghat
    temp_iqq2 = ds_BLta["ta"].sel(time=slice(datetime_begin, datetime_end)).mean(dim="time")
    axs[0].set_title("Temperature in ICON and Foghat IQQ\n average of 22nd and 23rd of August")
    axs[0].plot(temp_iqq2, ds_BLta["height"], label="temperature Foghat", color="blue")
    axs[0].plot(ds_icon_by_height["temp"].values, icon_heights, label="temperature ICON", color="red")
    axs[0].set_xlabel(ds_BLta["ta"].standard_name + " [" + ds_BLta["ta"].units + "]")
    axs[0].set_ylabel(ds_BLta["height"].standard_name + " [" + ds_BLta["height"].units + "]")
    axs[0].set_ylim(0, 2000)
    axs[0].set_xlim(270, 310)
    axs[0].legend(loc="upper left", fontsize=16)
    axs[0].grid(True)
    
    # Gesamtlayout und Speichern der Grafik
    plt.tight_layout()
    plt.savefig(outputdir + str(begin_float) + "_temp_comparison_subplots.png")
    
    plt.close("all")
    return 0

###############################################################
t_values = [20180822.0, 20180822.25, 20180822.5, 20180822.75, 20180823.0, 20180823.25, 20180823.5, 20180823.75,\
            20180824, 20180822.0,20180824, 20180822.625, 20180822.75, 20180822.125, 20180822.25, 20180823.125,\
            20180823.25, 20180823.625,\
            20180823.75]

for i, timestep in enumerate(t_values):
    if i+1==len(t_values):
        break

    if t_values[i+1]<timestep:
        print("overleaped timestep: ", timestep, " to ", t_values[i+1])
        continue
    
    begin_float = timestep
    end_float = t_values[i+1]
    print_all_temp_plots_for_timeperiod(begin_float=begin_float, end_float=end_float)

##############################
# Is time loc or UTC in both files?
# => I guess so!


# 2. draw errors!!!! in main plots als background color

overleaped timestep:  20180824  to  20180822.0
overleaped timestep:  20180824  to  20180822.625
overleaped timestep:  20180822.75  to  20180822.125


In [4]:
# Timeseries from measurement: (dry adiabatic correction from 10 to 2 m is required!)
# Make plots of temporal evolution of temperature and humidity here:

import xarray as xr
import matplotlib.pyplot as plt
import numpy as np
import sys
sys.path.append("/home/qwertz/Dokumente/N_eigene_Dokumente/Geo_Met_MSc/Semester_4/Thesis_prep_Juni_23/Porject_Work/")
from Sc_module import convert_single_timestamp
from Sc_module import convert_to_float_timestamps


print("Delete this if next run doesn't throw error")
'''
def convert_to_float_timestamps(timestamps):
    float_timestamps = []
    for ts in timestamps:
        ts_str = str(ts)
        day_str = ts_str[:10]
        hour = int(ts_str[11:13])
        minute = int(ts_str[14:16])
        second = int(ts_str[17:19])
        microsecond = int(ts_str[20:])
        
        # Berechnung des Float-Zeitstempels
        float_ts = float(day_str[:4])*10000+float(day_str[5:7])*100+float(day_str[8:10]) + \
                   (hour * 3600 + minute * 60 + second + microsecond * 1e-6) / 86400
        
        float_timestamps.append(float_ts)
    
    return np.array(float_timestamps)
'''
##################################################

file_hua22 = "/home/qwertz/obs_data_MA/iqq_foghat/22/sups_iqq_mwr00_l2_hua_p00_20180822000000.nc"
file_prw22 = "/home/qwertz/obs_data_MA/iqq_foghat/22/sups_iqq_mwr00_l2_prw_p00_20180822000000.nc"
file_00ta22 = "/home/qwertz/obs_data_MA/iqq_foghat/22/sups_iqq_mwr00_l2_ta_p00_20180822000000.nc"
file_BLta22 = "/home/qwertz/obs_data_MA/iqq_foghat/22/sups_iqq_mwrBL00_l2_ta_p00_20180822000243.nc"
file_hua23 = "/home/qwertz/obs_data_MA/iqq_foghat/23/sups_iqq_mwr00_l2_hua_p00_20180823000000.nc"
file_prw23 = "/home/qwertz/obs_data_MA/iqq_foghat/23/sups_iqq_mwr00_l2_prw_p00_20180823000000.nc"
file_00ta23 = "/home/qwertz/obs_data_MA/iqq_foghat/23/sups_iqq_mwr00_l2_ta_p00_20180823000000.nc"
file_BLta23 = "/home/qwertz/obs_data_MA/iqq_foghat/23/sups_iqq_mwrBL00_l2_ta_p00_20180823000753.nc"

ds_icon = xr.open_dataset("/home/qwertz/data_js_icon_22_23_August_2018/stat_IQQ_all-times.nc", engine="netcdf4")
ds_hua22 = xr.open_dataset(file_hua22)
ds_prw22 = xr.open_dataset(file_prw22)
ds_00ta22 = xr.open_dataset(file_00ta22)
ds_BLta22 = xr.open_dataset(file_BLta22)
ds_hua23 = xr.open_dataset(file_hua23)
ds_prw23 = xr.open_dataset(file_prw23)
ds_00ta23 = xr.open_dataset(file_00ta23)
ds_BLta23 = xr.open_dataset(file_BLta23)
ds_hua = xr.concat([ds_hua22, ds_hua23], dim="time") # absolute humidity kg m-3 rho
ds_prw = xr.concat([ds_prw22, ds_prw23], dim="time") # atmosphere_mass_content_of_water_vapor kg m-2 IWV
ds_00ta = xr.concat([ds_00ta22, ds_00ta23], dim="time") # T Kelvin
ds_BLta = xr.concat([ds_BLta22, ds_BLta23], dim="time") # t Kelvin / less timesteps...
#####
# Entfernen von doppelten Zeitpunkten:
ds_hua = ds_hua.drop_duplicates("time")
ds_prw = ds_prw.drop_duplicates("time")
ds_00ta = ds_00ta.drop_duplicates("time")
ds_BLta = ds_BLta.drop_duplicates("time")

##############################################################
# Prepare temp plot:

print("Foghat height: ", ds_00ta["height"].values[0])

####################################
# Die ICON-Daten stammen immernoch aus 59 Meter Höhe.... Da muss es auch welche von 10 m geben!!!
print("ICON height: ", ds_icon["z_mc"].values[199])
# print("ICON height: ", ds_icon["z_mc"])
####################################

begin_float = 20180822.0
end_float = 20180824.0
variable = "temp" # "rh", "qv", "qc", "qr"]
station = "IQQ"

# Interpolate icon data on foghat heights:
converted_foghat_times = convert_to_float_timestamps(ds_BLta["time"].values)
interp_func = interp1d(converted_foghat_times, ds_BLta["ta"].isel(height=0).values)
interpolated_foghat_temps = interp_func(ds_icon["time"].sel(time=slice(begin_float, end_float)).values)
#  ValueError: A value in x_new is below the interpolation range.
####
# Temperature Plot: 
x_values = [20180822.0, 20180822.25, 20180822.5, 20180822.75, 20180823.0, 20180823.25, 20180823.5, 20180823.75, 20180824]
        # [date for date in ds['date'].values[first_index::2000]]
xticks = ["08-22T00:00", "08-22T06:00", "08-22T12:00", "08-22T18:00", "08-23T00:00", "08-23T06:00",\
                  "08-23T12:00", "08-23T18:00", "08-24T00:00"]
plt.figure(figsize=(15,8))
plt.plot(ds_icon["time"].sel(time=slice(begin_float, end_float)).values,\
         ds_icon[variable].sel(time=slice(begin_float, end_float)).isel(height=199).isel(ncells=0).values, label="Temperature ICON")
plt.plot(ds["time"].sel(time=slice(begin_float, end_float)).values, interpolated_foghat_temps, label="Temperature Foghat")
plt.xticks(x_values, xticks)
plt.ylabel(variable+" in ["+ds[variable].units+"]")
plt.xlabel("time")
plt.xlim(20180822,20180824)
plt.ylim(270,310)
plt.title("Temporal evolution of "+variable+" at "+station+" for Foghat and ICON")
plt.legend()
plt.tight_layout()
plt.savefig("./Jans_Runs/Neue_plots/timeseries/"+str(begin_float)+"_temp_timeseries.png")
# plt.close()

#############################################################
# Add measurement dataXXX


# => Compare timeseries:
# dry adiabatic correction to two meters!
# Is time loc or UTC in both files?

Delete this if next run doesn't throw error
Foghat height:  0.0
ICON height:  [59.636677]


ValueError: A value in x_new is below the interpolation range.

In [ ]:
# humidity timeseries:

import xarray as xr
import matplotlib.pyplot as plt
import numpy as np
import sys
sys.path.append("/home/qwertz/Dokumente/N_eigene_Dokumente/Geo_Met_MSc/Semester_4/Thesis_prep_Juni_23/Porject_Work/")
from Sc_module import convert_single_timestamp
from Sc_module import convert_to_float_timestamps

######################
from Sc_module import abs_hum_from_rh

##################################################

file_hua22 = "/home/qwertz/obs_data_MA/iqq_foghat/22/sups_iqq_mwr00_l2_hua_p00_20180822000000.nc"
file_prw22 = "/home/qwertz/obs_data_MA/iqq_foghat/22/sups_iqq_mwr00_l2_prw_p00_20180822000000.nc"
file_00ta22 = "/home/qwertz/obs_data_MA/iqq_foghat/22/sups_iqq_mwr00_l2_ta_p00_20180822000000.nc"
file_BLta22 = "/home/qwertz/obs_data_MA/iqq_foghat/22/sups_iqq_mwrBL00_l2_ta_p00_20180822000243.nc"
file_hua23 = "/home/qwertz/obs_data_MA/iqq_foghat/23/sups_iqq_mwr00_l2_hua_p00_20180823000000.nc"
file_prw23 = "/home/qwertz/obs_data_MA/iqq_foghat/23/sups_iqq_mwr00_l2_prw_p00_20180823000000.nc"
file_00ta23 = "/home/qwertz/obs_data_MA/iqq_foghat/23/sups_iqq_mwr00_l2_ta_p00_20180823000000.nc"
file_BLta23 = "/home/qwertz/obs_data_MA/iqq_foghat/23/sups_iqq_mwrBL00_l2_ta_p00_20180823000753.nc"

ds_icon = xr.open_dataset("/home/qwertz/data_js_icon_22_23_August_2018/stat_IQQ_all-times.nc", engine="netcdf4")
ds_hua22 = xr.open_dataset(file_hua22)
ds_prw22 = xr.open_dataset(file_prw22)
ds_00ta22 = xr.open_dataset(file_00ta22)
ds_BLta22 = xr.open_dataset(file_BLta22)
ds_hua23 = xr.open_dataset(file_hua23)
ds_prw23 = xr.open_dataset(file_prw23)
ds_00ta23 = xr.open_dataset(file_00ta23)
ds_BLta23 = xr.open_dataset(file_BLta23)
ds_hua = xr.concat([ds_hua22, ds_hua23], dim="time") # absolute humidity kg m-3 rho
ds_prw = xr.concat([ds_prw22, ds_prw23], dim="time") # atmosphere_mass_content_of_water_vapor kg m-2 IWV
ds_00ta = xr.concat([ds_00ta22, ds_00ta23], dim="time") # T Kelvin
ds_BLta = xr.concat([ds_BLta22, ds_BLta23], dim="time") # t Kelvin / less timesteps...
#####
# Entfernen von doppelten Zeitpunkten:
ds_hua = ds_hua.drop_duplicates("time")
ds_prw = ds_prw.drop_duplicates("time")
ds_00ta = ds_00ta.drop_duplicates("time")
ds_BLta = ds_BLta.drop_duplicates("time")

##############################################################

def abs_hum_from_rh(rhi, tempi):
    e_pressure = vapour_pressure_e_from_rh(rhi, tempi)
    abs_hum = absolute_moisture_rhov_or_f(e_pressure, tempi)
    return abs_hum

'''
def convert_to_float_timestamps(timestamps):
    float_timestamps = []
    for ts in timestamps:
        ts_str = str(ts)
        day_str = ts_str[:10]
        hour = int(ts_str[11:13])
        minute = int(ts_str[14:16])
        second = int(ts_str[17:19])
        microsecond = int(ts_str[20:])
        
        # Berechnung des Float-Zeitstempels
        float_ts = float(day_str[:4])*10000+float(day_str[5:7])*100+float(day_str[8:10]) + \
                   (hour * 3600 + minute * 60 + second + microsecond * 1e-6) / 86400
        
        float_timestamps.append(float_ts)
    
    return np.array(float_timestamps)
'''
#####################################################

begin_float = 20180822.0
end_float = 20180824.0
variable = "rh" # "rh", "qv", "qc", "qr"]
station = "IQQ"

# Interpolate icon data on foghat heights:
converted_foghat_times = convert_to_float_timestamps(ds_hua["time"].values)
interp_func = interp1d(converted_foghat_times, ds_hua["hua"].isel(height=0).values) # ValueError: x and y arrays must be equal in length al axis
interpolated_foghat_temps = interp_func(ds_icon["time"].sel(time=slice(begin_float, end_float)).values)

# calculate abs hum from rh in ICON:
icon_rhs = ds_icon[variable].sel(time=slice(begin_float, end_float)).isel(height=199).isel(ncells=0).values
icon_temps = ds_icon["temp"].sel(time=slice(begin_float, end_float)).isel(height=199).isel(ncells=0).values
icon_to_plot = []
for i, (rhi,tempi) in enumerate(zip(icon_rhs, icon_temps)):
    icon_to_plot.append(abs_hum_from_rh(rhi, tempi))
    
####
# Humidity Plot: 
x_values = [20180822.0, 20180822.25, 20180822.5, 20180822.75, 20180823.0, 20180823.25, 20180823.5, 20180823.75, 20180824]
        # [date for date in ds['date'].values[first_index::2000]]
xticks = ["08-22T00:00", "08-22T06:00", "08-22T12:00", "08-22T18:00", "08-23T00:00", "08-23T06:00",\
                  "08-23T12:00", "08-23T18:00", "08-24T00:00"]
plt.figure(figsize=(15,8))
plt.plot(ds_icon["time"].sel(time=slice(begin_float, end_float)).values,icon_to_plot, label="Temperature ICON")
plt.plot(ds["time"].sel(time=slice(begin_float, end_float)).values, interpolated_foghat_temps, label="Temperature Foghat")
plt.xticks(x_values, xticks)
plt.ylabel("qv in [kg m-2]")
plt.xlabel("time")
plt.xlim(20180822,20180824)
plt.ylim(0,0.011)
plt.title("Temporal evolution of Absolute Humidity at "+station+" for Foghat and ICON")
plt.legend()
plt.tight_layout()
plt.savefig("./Jans_Runs/Neue_plots/timeseries/"+str(begin_float)+"_humidity_timeseries.png")
# plt.close()

####################################################


# Fix a color code: one color for ICON in all plots, one color for differences and biases....
# Maybe also rh plot?
# Eher diese für Vergleich: 
#dmc = daten des chilenischen wetterdienstes: 2m wind (ff), Temperatur (ta), feuchte (rh und td)
#             und ceilometer (ceilo --> wolkenuntergrenzen, bedeckungsgrad ... details müs)